# PROJECT-5. Эффективность каналов привлечения и воронка продаж
### Пархоменко Евгения Александровна
#### DARP 1018

### ЦЕЛЬ
Проанализировать эффективность каналов привлечения и воронка продаж.

### Ход проверки

Импортируем нужные библиотеки

In [7]:
import pandas as pd
import numpy as np
import psycopg2.extras
import seaborn as sns
from collections import Counter
import itertools
from itertools import combinations
import plotly
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import matplotlib.pyplot as plt


In [27]:
data_df = pd.read_csv("C:\IDE\Dataset\SF_case_data.csv")
data = data_df.copy()
data.head(3)

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal


In [33]:
data1 = data.groupby('date')['event'].count().reset_index()
print(data1.sort_values(by='event', ascending=False))

          date  event
9   2020-01-10  89831
65  2020-03-06  63264
10  2020-01-11  58169
44  2020-02-14  54233
13  2020-01-14  53783
..         ...    ...
5   2020-01-06  19717
3   2020-01-04  19152
90  2020-03-31  19003
7   2020-01-08  18616
6   2020-01-07  18226

[91 rows x 2 columns]


In [34]:
mask = data['event'] == 'app_install'
mask1 = data['date'] == '2020-03-31' 
print(data[mask & mask1])

               date        event  purchase_sum  os_name  device_id  gender  \
2668690  2020-03-31  app_install           NaN      ios    2982559    male   
2668691  2020-03-31  app_install           NaN      ios     207020  female   
2668692  2020-03-31  app_install           NaN      ios   26794869  female   
2668693  2020-03-31  app_install           NaN      ios    7476982  female   
2668694  2020-03-31  app_install           NaN  android   18709025  female   
...             ...          ...           ...      ...        ...     ...   
2669653  2020-03-31  app_install           NaN      ios   19717968  female   
2669654  2020-03-31  app_install           NaN  android   32966665  female   
2669655  2020-03-31  app_install           NaN      ios    6335964  female   
2669656  2020-03-31  app_install           NaN  android   29155826    male   
2669657  2020-03-31  app_install           NaN      ios   35057676    male   

                     city     utm_source  
2668690            M

In [35]:
installs = data[data['event'] == 'app_install']
data['install_date'] = data['device_id'].map(installs.set_index('device_id')['date'])
def new_column(date1,date2,value):
    if date1 == date2:
        return value
data['install_group'] = data.apply(lambda x: new_column(date1 = x['date'], date2 = x['install_date'], value = 'Установили в этот же день'), axis=1)
data['install_group'] = data['install_group'].fillna('Остальные')
df2 = data[data['event'] == 'app_start']['install_group'].value_counts()
df3 = data[data['event'] == 'app_start'].groupby(['date','install_group']).size().reset_index(name = 'users')
print(df3[df3['date'] == '2020-02-14'])

          date              install_group  users
88  2020-02-14                  Остальные  10282
89  2020-02-14  Установили в этот же день   4188


In [36]:
# Сначала мы находим дату, когда были установлены приложения:
installs = data[data['event'] == 'app_install']
data['install_date'] = data['device_id'].map(installs.set_index('device_id')['date'])

In [37]:
# После этого мы пишем функцию, которая покажет, какие пользователи «Установили в тот же день» и применяем ее к нашему датафрейму:
def new_column(date1,date2,value):
    if date1 == date2:
        return value
data['install_group'] = data.apply(lambda x: new_column(date1 = x['date'], date2 = x['install_date'], value = 'Установили в этот же день'), axis=1)

In [38]:
# Далее заполняем пропуски в колонке ‘install_group’ и заменяем их на ‘Остальные’.
data['install_group'] = data['install_group'].fillna('Остальные')

In [39]:
# После чего мы создаем новый датафрейм, чтобы отобразить динамику по дням:
df = data[data['event'] == 'app_start'].groupby(['date','install_group']).size().reset_index(name = 'users')

In [40]:
# В данном задании сначала нам нужно воспользоваться нашим датафреймом data и выделить оттуда пользователей, которые открыли приложение:
traffic=data[data['event']=='app_start']

In [41]:
# Потом мы создаем два новых датафрейма, где считаем количество инсталлов/установок (app_install) и количество открытий по дням:
same_day= installs.groupby(['date'])['device_id'].count().reset_index()
total= traffic.groupby(['date'])['device_id'].count().reset_index()

In [42]:
# Потом мы соединяем оба датафрейма в один и переименовываем столбцы:
app_open_perc=same_day.merge(total,how='left',on='date')
app_open_perc.rename(columns={'device_id_x':'same_day', 'device_id_y': 'total'},inplace=True)

In [43]:
# Далее мы вычитаем из общего количества количество тех, кто установил в тот же день:
app_open_perc['other']=app_open_perc['total']-app_open_perc['same_day']

In [46]:
# И на последнем этапе находим долю открытий приложений:
app_open_perc['percent']=round(app_open_perc['other']/app_open_perc['total']*100)
print(app_open_perc['percent'])

0     19.0
1     38.0
2     53.0
3     63.0
4     70.0
      ... 
86    85.0
87    83.0
88    81.0
89    83.0
90    82.0
Name: percent, Length: 91, dtype: float64


In [28]:
# Сначала вы подготавливаете две таблицы. Одна с инсталлами, другая с открытиями:
traffic=data[data['event']=='app_start']
installs=data[data['event']=='app_install']

In [29]:
# Далее группируем эти таблицы по дням и считаем количество устройств:
same_day= installs.groupby(['date'])['device_id'].count().reset_index()
total= traffic.groupby(['date'])['device_id'].count().reset_index()

In [30]:
# После этого мы соединяем эти таблицы и переименовываем колонки для больше наглядности:
app_open_perc=same_day.merge(total,how='left',on='date')
app_open_perc.rename(columns={'device_id_x':'same_day', 'device_id_y': 'total'},inplace=True)

In [31]:
# Потом вам нужно найти разницу и посчитать процент:
app_open_perc['other']=app_open_perc['total']-app_open_perc['same_day']
app_open_perc['percent']=round(app_open_perc['other']/app_open_perc['total']*100)
app_open_perc

,date,same_day,total,other,percent
0,2020-01-01,3579,4445,866,19.0
1,2020-01-02,3144,5101,1957,38.0
2,2020-01-03,2402,5144,2742,53.0
3,2020-01-04,1831,4924,3093,63.0
4,2020-01-05,1671,5607,3936,70.0
...,...,...,...,...,...
86,2020-03-27,1199,7809,6610,85.0
87,2020-03-28,1091,6465,5374,83.0
88,2020-03-29,1117,5935,4818,81.0
89,2020-03-30,994,5704,4710,83.0


In [47]:
# Создадим два отдельных датафрейма:
gr1 = df[df['install_group'] == 'Остальные']
gr2 = df[df['install_group'] != 'Остальные']

In [48]:
# Объединяем новые датафреймы по дате:
stat = gr1.merge(gr2, how = 'left', on = 'date')

In [49]:
# Считаем долю трафика от инсталлов/установок по дням и находим, в какой день она наибольшая:
stat['Доля трафика с инсталлов, %'] = stat['users_y']/(stat['users_y'] +stat['users_x'])
stat.sort_values('Доля трафика с инсталлов, %', ascending = True).head()

,date,install_group_x,users_x,install_group_y,users_y,"Доля трафика с инсталлов, %"
75,2020-03-16,Остальные,5786,Установили в этот же день,360,0.058575
76,2020-03-17,Остальные,6111,Установили в этот же день,431,0.065882
64,2020-03-05,Остальные,5759,Установили в этот же день,411,0.066613
73,2020-03-14,Остальные,6166,Установили в этот же день,443,0.067030
61,2020-03-02,Остальные,6015,Установили в этот же день,453,0.070037


In [51]:
# Выбираем пользователей, которые установили приложение и их группируем:
df_install = data[data['event'] == 'app_install'].groupby(['date','utm_source']).size().reset_index(name = 'users')

In [53]:
# Считаем суммарное количество пользователей пришедших с разных каналов по дням:
df_install = pd.pivot_table(df_install, values='users', index='utm_source', aggfunc='sum').sort_values('users',ascending = False)
print(df_install)

               users
utm_source          
-              32460
yandex-direct  29368
google_ads     26286
vk_ads         23189
instagram_ads  20096
facebook_ads   13916
referal         9282


In [54]:
# выбираем пользователей, которые открыли приложение и группируем по дате и каналу, с которого эти пользователи пришли:
df_app_start = data[data['event'] == 'app_start'].drop_duplicates('device_id').groupby(['date','utm_source']).size().reset_index(name = 'users')

In [56]:
df_app_start = pd.pivot_table(df_app_start, values='users', index='utm_source', aggfunc='sum').sort_values('users',ascending = False)
print(df_app_start)

               users
utm_source          
-              41456
yandex-direct  34441
google_ads     31437
vk_ads         27905
instagram_ads  24818
facebook_ads   18844
referal        11983


In [57]:
first_registration = data[data['event'] == 'register'].sort_values('date').drop_duplicates('device_id')
first_registration['finally_registered'] = 1

In [58]:
data['first_registration'] = data['device_id'].map(first_registration.set_index('device_id')['date'])
data['finally_registered'] = data['device_id'].map(first_registration.set_index('device_id')['finally_registered'])

In [59]:
already_reg = data[data['first_registration'] < data['date']]
already_reg['registered'] = 1
requires_reg1 = data[(data['first_registration'].isna() == True)]

In [60]:
requires_reg1['registered'] = 0
requires_reg2 =  data[data['first_registration'] >= data['date']]
requires_reg2['registered'] = 0

In [61]:
data = pd.concat([already_reg,requires_reg1,requires_reg2])
data[data['event'] == 'tap_basket'].groupby('registered').count()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,install_date,install_group,first_registration,finally_registered
registered,,,,,,,,,,,,
0,160409,160409,0,160409,160409,160409,160409,160409,130022,160409,105031,105031
1,217256,217256,0,217256,217256,217256,217256,217256,177108,217256,217256,217256


In [62]:
reg = data[data['registered'] == 1]
funnel_reg = pd.crosstab(reg.utm_source, reg.event, values=reg.device_id, aggfunc=pd.Series.nunique)
funnel_reg

event,app_start,choose_item,purchase,search,tap_basket
utm_source,,,,,
-,9482,8728,5492,9355,7850
facebook_ads,5606,5154,3333,5527,4694
google_ads,5399,4948,3121,5335,4438
instagram_ads,5686,5288,3548,5618,4842
referal,3647,3417,2382,3610,3155
vk_ads,6011,5636,3950,5958,5196
yandex-direct,5396,4928,3099,5304,4464


In [66]:
funnel_reg['1 step'] = funnel_reg['search']*100/funnel_reg['app_start']
funnel_reg['2 step'] = funnel_reg['choose_item']*100/funnel_reg['search']
funnel_reg['3 step'] = funnel_reg['tap_basket']*100/funnel_reg['choose_item']
funnel_reg['5 step'] = funnel_reg['purchase']*100/funnel_reg['tap_basket']
funnel_reg

event,app_start,choose_item,purchase,search,tap_basket,1 step,2 step,3 step,5 step
utm_source,,,,,,,,,
-,9482,8728,5492,9355,7850,98.660620,93.297702,89.940422,69.961783
facebook_ads,5606,5154,3333,5527,4694,98.590796,93.251312,91.074893,71.005539
google_ads,5399,4948,3121,5335,4438,98.814595,92.746017,89.692805,70.324470
instagram_ads,5686,5288,3548,5618,4842,98.804080,94.126023,91.565809,73.275506
referal,3647,3417,2382,3610,3155,98.985468,94.653740,92.332455,75.499208
vk_ads,6011,5636,3950,5958,5196,99.118283,94.595502,92.193045,76.020015
yandex-direct,5396,4928,3099,5304,4464,98.295033,92.911011,90.584416,69.422043


In [68]:
reg1 = data[data['registered'] == 0]
funnel_reg1 = pd.crosstab(reg.utm_source, reg.event, values=reg.device_id, aggfunc=pd.Series.nunique)

In [69]:
funnel_reg1['1 step'] = funnel_reg1['search']*100/funnel_reg1['app_start']
funnel_reg1['2 step'] = funnel_reg1['choose_item']*100/funnel_reg1['search']
funnel_reg1['3 step'] = funnel_reg1['tap_basket']*100/funnel_reg1['choose_item']
funnel_reg1['5 step'] = funnel_reg1['purchase']*100/funnel_reg1['tap_basket']
funnel_reg1

event,app_start,choose_item,purchase,search,tap_basket,1 step,2 step,3 step,5 step
utm_source,,,,,,,,,
-,9482,8728,5492,9355,7850,98.660620,93.297702,89.940422,69.961783
facebook_ads,5606,5154,3333,5527,4694,98.590796,93.251312,91.074893,71.005539
google_ads,5399,4948,3121,5335,4438,98.814595,92.746017,89.692805,70.324470
instagram_ads,5686,5288,3548,5618,4842,98.804080,94.126023,91.565809,73.275506
referal,3647,3417,2382,3610,3155,98.985468,94.653740,92.332455,75.499208
vk_ads,6011,5636,3950,5958,5196,99.118283,94.595502,92.193045,76.020015
yandex-direct,5396,4928,3099,5304,4464,98.295033,92.911011,90.584416,69.422043


In [70]:
first_opens = data[data['event'] == 'app_start'].sort_values('date').drop_duplicates('device_id').groupby(['utm_source']).size().reset_index(name = 'users')
first_purchase = data[data['event'] == 'purchase'].sort_values('date').drop_duplicates('device_id').groupby(['utm_source']).size().reset_index(name = 'users')

stat = first_opens.merge(first_purchase, how = 'left', on = 'utm_source')
stat['CR to first purchase'] = stat['users_y']/stat['users_x']
stat

,utm_source,users_x,users_y,CR to first purchase
0,-,41456,14786,0.356667
1,facebook_ads,18844,7903,0.419391
2,google_ads,31437,10167,0.323409
3,instagram_ads,24818,9820,0.395681
4,referal,11983,5803,0.484269
5,vk_ads,27905,11460,0.410679
6,yandex-direct,34441,10936,0.317529


In [72]:
first_purchase = data[data['event'] == 'purchase'].sort_values('date').drop_duplicates('device_id').groupby('utm_source')['purchase_sum'].median()
first_purchase

utm_source
-                398.5
facebook_ads     389.0
google_ads       390.5
instagram_ads    393.5
referal          395.5
vk_ads           393.0
yandex-direct    392.5
Name: purchase_sum, dtype: float64

In [73]:
# Переводим данные в формат даты:
data['date'] = pd.to_datetime(data['date'])

In [74]:
# Находим неделю первого посещения и делаем новую таблицу на основе data с данными только о начале использования приложения (app_start)
entrances = data[data['event'] == 'app_start'].sort_values('date').drop_duplicates('device_id') 

In [75]:
# В таблице entrances устанавливаем индекс device_id:
entrances.set_index('device_id')

,date,event,purchase_sum,os_name,gender,city,utm_source,install_date,install_group,first_registration,finally_registered,registered
device_id,,,,,,,,,,,,
198435,2020-01-01,app_start,NaN,android,male,Saint-Petersburg,vk_ads,2020-01-01,Установили в этот же день,2020-01-03,1.0,0
9279324,2020-01-01,app_start,NaN,android,male,Moscow,yandex-direct,2020-01-01,Установили в этот же день,2020-01-01,1.0,0
9069447,2020-01-01,app_start,NaN,ios,male,Saint-Petersburg,instagram_ads,2020-01-01,Установили в этот же день,2020-01-06,1.0,0
8058489,2020-01-01,app_start,NaN,ios,female,Moscow,-,NaN,Остальные,2020-01-04,1.0,0
7105494,2020-01-01,app_start,NaN,android,male,Moscow,yandex-direct,NaN,Остальные,2020-01-01,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
29313543,2020-03-31,app_start,NaN,android,female,Moscow,facebook_ads,2020-03-31,Установили в этот же день,NaN,NaN,0
22501429,2020-03-31,app_start,NaN,android,male,Saint-Petersburg,facebook_ads,NaN,Остальные,NaN,NaN,0
2041985,2020-03-31,app_start,NaN,ios,female,Moscow,instagram_ads,2020-03-31,Установили в этот же день,NaN,NaN,0


In [76]:
# Делаем серию из датафрейма entrances с device_id, который является индексом, и столбцом date:
entrances.set_index('device_id')['date']

device_id
198435     2020-01-01
9279324    2020-01-01
9069447    2020-01-01
8058489    2020-01-01
7105494    2020-01-01
              ...    
29313543   2020-03-31
22501429   2020-03-31
2041985    2020-03-31
34124302   2020-03-31
908531     2020-03-31
Name: date, Length: 190884, dtype: datetime64[ns]

In [77]:
# Применяем это действие к device_id из датафрейма data по индексу device_id из датафрейма entrances:
data['device_id'].map(entrances.set_index('device_id')['date'])

15148     2020-01-01
15175     2020-01-01
15197     2020-01-01
15210     2020-01-01
15214     2020-01-01
             ...    
2747963   2020-03-28
2747964   2020-03-31
2747965   2020-03-31
2747966   2020-03-06
2747967   2020-03-31
Name: device_id, Length: 2747968, dtype: datetime64[ns]

In [78]:
# Добавляем новый столбец 'first_entrance' в датафрейм data:
data['first_entrance'] = data['device_id'].map(entrances.set_index('device_id')['date'])